In [38]:
from copy import deepcopy

import torch

In [39]:
torch.manual_seed(0);

In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [41]:
from senmodel.model.utils import *

In [42]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=8):
        super(SimpleFCN, self).__init__()
        self.fc1 = nn.Linear(input_size, 4)
        self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(2, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = SimpleFCN()

In [43]:
sparse_model = convert_dense_to_sparse_network(model)

In [44]:
linear = nn.Linear(8, 1)

sparse_weight = dense_to_sparse(linear.weight.data)
sparse_bias = dense_to_sparse(linear.bias.data)
just_sparse_linear = ExpandingLinear(sparse_weight, sparse_bias)

sparse_linear = deepcopy(just_sparse_linear)

print(sparse_linear.weight_indices, "\n")
sparse_linear.replace_many((0, 0), (6, 7))
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[0].weight_indices, "\n")

sparse_linear.replace_many((0, 0), (8, 9))
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[1].weight_indices, "\n")

print(sparse_linear.weight_indices)
sparse_linear.replace_many((0,), (10,))
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[-1].weight_indices, "\n")


# sparse_linear.replace_many((0,), (10,)) # error - no such edge
# print(sparse_linear.weight_indices)
# print(sparse_linear.embed_linears[-1].weight_indices, "\n")



tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4, 5, 6, 7]]) 

tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4, 5, 8, 9]])
tensor([[0, 1],
        [6, 7]]) 

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4,  5, 10, 11]])
tensor([[0, 1],
        [8, 9]]) 

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4,  5, 10, 11]])
tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4,  5, 11, 12]])
tensor([[ 0],
        [10]]) 



In [45]:
from senmodel.metrics.nonlinearity_metrics import *


In [46]:
criterion = nn.MSELoss()

metrics = [
    GradientMeanEdgeMetric(criterion),
    PerturbationSensitivityEdgeMetric(criterion),
]

In [47]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=1000, n_features=8, n_informative=8, random_state=42)
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()

dataset = list(zip(X, y))
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [48]:

from senmodel.metrics.edge_finder import *

ef = EdgeFinder(metrics[0], test_loader, device=device)

In [49]:
print("values:", ef.calculate_edge_metric_for_dataloader(sparse_linear))
chosen_edges = ef.choose_edges_top_k(sparse_linear, 4)
print("choose:", chosen_edges)

chosen_edges = ef.choose_edges_threshold(sparse_linear, 0.1)
print("choose:", chosen_edges)



values: tensor([7.6116e-01, 2.9361e-01, 2.7699e-01, 6.8202e-01, 3.1025e-01, 1.6711e-01,
        6.4410e-02, 6.1011e-04])
choose: tensor([[0, 0, 0, 0],
        [0, 3, 4, 1]])
choose: tensor([[0, 0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4, 5]])


In [50]:
sparse_linear.weight_indices

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4,  5, 11, 12]])

In [51]:
sparse_linear.replace_many(*chosen_edges)

In [52]:
sparse_linear.weight_indices

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [11, 12, 13, 14, 15, 16, 17, 18]])

In [53]:
print("values:", ef.calculate_edge_metric_for_dataloader(sparse_linear))
chosen_edges = ef.choose_edges_top_k(sparse_linear, 5)
print("choose:", chosen_edges)

values: tensor([0.4667, 0.0309, 0.0623, 0.4901, 0.2237, 0.0408, 0.3938, 0.6195])
choose: tensor([[ 0,  0,  0,  0,  0],
        [18, 14, 11, 17, 15]])


In [54]:
sparse_linear.replace_many(*chosen_edges)

In [55]:
sparse_linear.weight_indices

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [12, 13, 16, 19, 20, 21, 22, 23]])

In [56]:
print("values:", ef.calculate_edge_metric_for_dataloader(sparse_linear))
chosen_edges = ef.choose_edges_top_k(sparse_linear, 4)
print("choose:", chosen_edges)

values: tensor([0.1435, 0.2855, 0.1623, 0.7250, 0.0698, 0.5946, 0.3446, 0.0183])
choose: tensor([[ 0,  0,  0,  0],
        [19, 21, 22, 13]])
